#  Forward pass

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Deep Learning with PyTorch (2020) Авторы: Eli Stevens, Luca Antiga, Thomas Viehmann
* https://pytorch.org/docs/stable/generated/torch.matmul.html
* https://machinelearningmastery.com/choose-an-activation-function-for-deep-learning/
* https://machinelearningmastery.com/loss-and-loss-functions-for-training-deep-learning-neural-networks/

## Задачи для совместного разбора

1\. Используя операции над матрицами и векторами из библиотеки `torch`, реализуйте нейрон с заданными весами `weights` и `bias`. Пропустите вектор `inputs` через нейрон и выведите результат.

In [46]:
import torch as th

In [16]:
class Neuron:
  def __init__(self, weights, bias):
    # <создать атрибуты объекта weights и bias>
    self.weights = weights
    self.bias = bias

  def forward(self, inputs):
    return inputs @ self.weights + self.bias

In [17]:
inputs = torch.tensor([1.0, 2.0, 3.0, 4.0])
weights = torch.tensor([-0.2, 0.3, -0.5, 0.7])
bias = 3.14

In [18]:
neuron = Neuron(weights, bias)
y_pred = neuron.forward(inputs)
y_pred

tensor(4.8400)

2\. Используя операции над матрицами и векторами из библиотеки `torch`, реализуйте функцию активации ReLU:

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/f4353f4e3e484130504049599d2e7b040793e1eb)

Создайте матрицу размера (4,3), заполненную числами из стандартного нормального распределения, и проверьте работоспособность функции активации.

In [19]:
class ReLU:
  def forward(self, inputs):
    # <реализовать логику ReLU>
    return torch.clip(inputs, min=0)

In [20]:
X = torch.randn((4, 3))
X

tensor([[ 0.5985, -0.4014, -0.7405],
        [-0.7314, -1.3956, -0.9422],
        [ 0.2657,  0.1320,  0.7004],
        [ 1.4247,  1.9337, -1.0005]])

In [21]:
relu = ReLU()
relu.forward(X)

tensor([[0.5985, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000],
        [0.2657, 0.1320, 0.7004],
        [1.4247, 1.9337, 0.0000]])

3\. Используя операции над матрицами и векторами из библиотеки `torch`, реализуйте функцию потерь MSE:

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/e258221518869aa1c6561bb75b99476c4734108e)
где $Y_i$ - правильный ответ для примера $i$, $\hat{Y_i}$ - предсказание модели для примера $i$, $n$ - количество примеров в батче.

In [22]:
class MSELoss:
  def forward(self, y_pred, y_true):
    return (y_pred - y_true).pow(2).mean()

In [23]:
y_pred = torch.tensor([1.0, 3.0, 5.0])
y_true = torch.tensor([2.0, 3.0, 4.0])

In [24]:
criterion = MSELoss()
criterion.forward(y_pred, y_true)

tensor(0.6667)

## Задачи для самостоятельного решения

### Cоздание полносвязных слоев

<p class="task" id="1_1"></p>

1\.1 Используя операции над матрицами и векторами из библиотеки `torch`, реализуйте полносвязный слой из `n_neurons` нейронов с `n_features` весами у каждого нейрона (инициализируются из стандартного нормального распределения) и опциональным вектором смещения.

$$y = xW^T + b$$

Пропустите вектор `inputs` через слой и выведите результат. Результатом прогона сквозь слой должна быть матрица размера `batch_size` x `n_neurons`.

- [x] Проверено на семинаре

In [25]:
class Linear:
    def __init__(self, n_neurons, n_features, bias: bool = False):
      self.weights = torch.randn(n_neurons, n_features)
      if bias:
        self.bias=torch.randn(1,n_neurons)
      else:
        self.bias=None


    def forward(self, inputs):
      outputs = inputs @ self.weights.t()
      if self.bias is not None:
        outputs+=self.bias
      return outputs

In [26]:
# inputs: batch_size(3) x 4
inputs = torch.tensor([[1, 2, 3, 2.5],
                       [2, 5, -1, 2],
                       [-1.5, 2.7, 3.3, -0.8]])

In [27]:
neuron = Linear(n_neurons=3, n_features=4)
y_pred = neuron.forward(inputs)
y_pred

tensor([[ 1.7992, -6.2152,  4.7225],
        [-4.9013, -2.4851,  7.8859],
        [-0.1520,  1.5608, -6.9860]])

<p class="task" id="1_2"></p>

1\.2 Используя решение предыдущей задачи, создайте 2 полносвязных слоя и пропустите тензор `inputs` последовательно через эти два слоя. Количество нейронов в первом слое выберите произвольно, количество нейронов во втором слое выберите так, чтобы результатом прогона являлась матрица `batch_size x 7`.

- [x] Проверено на семинаре

In [39]:
a1 = Linear(9,4, True)
b1 = a1.forward(inputs)

In [40]:
a2 = Linear(5,9)
b2 = a2.forward(b1)
b2

tensor([[ -2.4248,  -8.9219, -13.8766,  20.0407, -14.3747],
        [-13.4613, -18.6408,   9.3898,   0.5431,  -9.1883],
        [ 18.6718,  -8.5347,   1.9302,  17.5572, -20.7366]])

In [41]:
f2.shape

torch.Size([3, 5])

### Создание функций активации

<p class="task" id="2_1"></p>

2\.1 Используя операции над матрицами и векторами из библиотеки `torch`, реализуйте функцию активации softmax:

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/6d7500d980c313da83e4117da701bf7c8f1982f5)

$$\overrightarrow{x} = (x_1, ..., x_J)$$

Создайте матрицу размера (4,3), заполненную числами из стандартного нормального распределения, и проверьте работоспособность функции активации. Строки матрицы трактовать как выходы линейного слоя некоторого классификатора для 4 различных примеров. Функция должна применяться переданной на вход матрице построчно.

- [ ] Проверено на семинаре

In [49]:
class Softmax:
    def forward(self, inputs):
        exp_inputs = th.exp(inputs)
        return (exp_inputs / exp_inputs.sum(dim=1, keepdim=True))

In [50]:
sofm = Softmax()
sofm.forward(inputs)

tensor([[0.0641, 0.1744, 0.4740, 0.2875],
        [0.0452, 0.9074, 0.0022, 0.0452],
        [0.0052, 0.3488, 0.6355, 0.0105]])

In [51]:
th.exp(inputs[0]) / th.exp(inputs[0]).sum()

tensor([0.0641, 0.1744, 0.4740, 0.2875])

In [52]:
th.exp(inputs[1]) / th.exp(inputs[1]).sum()

tensor([0.0452, 0.9074, 0.0022, 0.0452])

In [57]:
sofm.forward(inputs).sum(axis=1)

tensor([1.0000, 1.0000, 1.0000])

<p class="task" id="2_2"></p>

2.2 Используя операции над матрицами и векторами из библиотеки `torch`, реализуйте функцию активации ELU:

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/eb23becd37c3602c4838e53f532163279192e4fd)

Создайте матрицу размера 4x3, заполненную числами из стандартного нормального распределения, и проверьте работоспособность функции активации.

- [ ] Проверено на семинаре

In [ ]:
class ELU:
    def __init__(self, alpha: float):
        pass

    def forward(self, inputs):
        # <реализовать логику ELU>
        pass

### Создание функции потерь

<p class="task" id="3_1"></p>

3\.1 Используя операции над матрицами и векторами из библиотеки `torch`, реализуйте функцию потерь CrossEntropyLoss:

$$y_i = (y_{i,1},...,y_{i,k})$$
<img src="https://i.ibb.co/93gy1dN/Screenshot-9.png" width="200">

$$ CrossEntropyLoss = \frac{1}{n}\sum_{i=1}^{n}{L_i}$$
где $y_i$ - вектор правильных ответов для примера $i$, $\hat{y_i}$ - вектор предсказаний модели для примера $i$; $k$ - количество классов, $n$ - количество примеров в батче.

Создайте полносвязный слой с 3 нейронами и прогнать через него батч `inputs`. Полученный результат пропустите через функцию активации Softmax. Посчитайте значение функции потерь, трактуя вектор `y` как вектор правильных ответов.

- [ ] Проверено на семинаре

In [66]:
class CrossEntropyLoss:
  def forward(self, y_pred, y_true):
    res = - (y_true * th.log(y_pred)).mean()
    return res

In [67]:
inputs = torch.tensor([[1, 2, 3, 2.5],
                        [2, 5, -1, 2],
                        [-1.5, 2.7, 3.3, -0.8]])
y = torch.tensor([1, 0, 0])

In [72]:
a1 = Linear(4,4, True)
b1 = a1.forward(inputs)

a2 = Linear(4,4)
b2 = a2.forward(b1)

a3 = Linear(2,4)
b3 = a3.forward(b2)

sofm = Softmax()
y_pred = sofm.forward(b3)
y_pred

tensor([[1.0000e+00, 1.0123e-09],
        [1.0000e+00, 1.4979e-10],
        [1.0000e+00, 4.2803e-06]])

In [73]:
CEL = CrossEntropyLoss()
CEL.forward(y_pred[:,1], y)

tensor(6.9037)

<p class="task" id="3_2"></p>

3.2 Модифицируйте MSE, добавив L2-регуляризацию.

$$MSE_R = MSE + \lambda\sum_{i=1}^{m}w_i^2$$

где $\lambda$ - коэффициент регуляризации; $w_i$ - веса модели.

- [ ] Проверено на семинаре

In [ ]:
class MSERegularized:
    def __init__(self, lambda_):
        pass

    def data_loss(self, y_pred, y_true):
        # <подсчет первого слагаемого из формулы>
        pass

    def reg_loss(self, weights):
        # <подсчет второго слагаемого из формулы>
        pass

    def forward(self, y_pred, y_true, weights):
        return self.data_loss(y_pred, y_true) + self.reg_loss(weights)

In [ ]:
y_pred = torch.tensor([-0.5, 1, 1.7])
y_true = torch.tensor([0, 0.6, 2.3])
weights = torch.normal(0, 5, (10, 1))

## Обратная связь
- [ ] Хочу получить обратную связь по решению